In [1]:
import VariationalBayes as vb
from VariationalBayes.SparseObjectives import SparseObjective, Objective
import VariationalBayes.ExponentialFamilies as ef
import VariationalBayes.Modeling as modeling

import math

import autograd
import autograd.numpy as np
import numpy as onp

import matplotlib.pyplot as plt
%matplotlib inline

import time

from copy import deepcopy
import scipy as sp
from scipy import optimize
from scipy import stats

In [2]:
# This is a variational implementation of the hierarchical_2pl model from the Stan examples.
np.random.seed(42)

num_i = 10
num_j = 50

true_params = vb.ModelParamsDict('true')
true_params.push_param(vb.VectorParam('alpha', size=num_i, lb=0))
true_params.push_param(vb.VectorParam('beta', size=num_i))
true_params.push_param(vb.VectorParam('theta', size=num_j))
true_params.push_param(vb.VectorParam('mu', size=2))
true_params.push_param(vb.PosDefMatrixParam('sigma', size=2))

prior_params = vb.ModelParamsDict('prior')
prior_params.push_param(vb.VectorParam('mu_mean', size=2, val=np.array([0., 0.])))
prior_params.push_param(vb.PosDefMatrixParam('mu_cov', size=2, val=np.array([[1., 0.], [0., 25.]])))
prior_params.push_param(vb.VectorParam('tau_param', size=2, val=np.array([0.1, 0.1])))
prior_params.push_param(vb.ScalarParam('lkj_param', val=4.))


In [3]:
# Set true parameters and generate data.

# y is defined as a num_i * num_j matrix.  Here, alpha and beta are num_i vectors and
# theta is the num_j vector.  Practically we expect num_j >> num_i.
# Combine vectors in the appropriate way to match the shape of y.
def get_logit_p_term(alpha, beta, theta):
    return (np.expand_dims(theta, 0) - np.expand_dims(beta, 1)) * np.expand_dims(alpha, 1)

true_params['alpha'].set(np.exp(np.random.random(num_i)))
true_params['beta'].set(np.random.random(num_i) - 0.5)
true_params['theta'].set(np.random.random(num_j) - 0.5)
true_params['mu'].set(np.random.random(2))
true_params['sigma'].set(np.eye(2))

logit_p = get_logit_p_term(alpha=true_params['alpha'].get(),
                           beta=true_params['beta'].get(),
                           theta=true_params['theta'].get())
y_prob = sp.special.expit(logit_p)
y = sp.stats.bernoulli.rvs(y_prob)
print(y.shape)


(10, 50)


In [5]:
vb_params = vb.ModelParamsDict('params')
vb_params.push_param(vb.UVNParamVector('log_alpha', length=num_i))
vb_params.push_param(vb.UVNParamVector('beta', length=num_i))
vb_params.push_param(vb.UVNParamVector('theta', length=num_j))
vb_params.push_param(vb.MVNParam('mu', dim=2))
vb_params.push_param(vb.WishartParam('sigma', size=2))


AttributeError: module 'VariationalBayes.Parameters' has no attribute 'PosDefMatrixParam'

In [ ]:
class Objective(object):
    def __init__(y, vb_params, prior_params, num_draws):
        self.y = deepcopy(y)
        self.vb_params = deepcopy(vb_params)
        self.prior_params = deepcopy(prior_params)
        self.std_draws = modeling.get_standard_draws(num_draws)

    def e_log_data_likelihood():
        # P(y = 1) = expit(z)
        log_alpha = self.vb_params['log_alpha']
        beta = self.vb_params['beta']
        theta = self.vb_params['theta']

        e_z = get_logit_p_term(alpha=log_alpha.e_exp(), beta=beta.e(), theta=theta.e())
        var_z = get_logit_p_term(alpha=log_alpha.e2_exp(), beta=-1 * beta.var(), theta=theta.var())